In [1]:
!pip install transformers<=4.28.0
!pip install sentencepiece
!pip install gdown

/bin/bash: =4.28.0: No such file or directory


In [2]:
import json
import os
from typing import Union,List
import sys

import torch
from transformers import XGLMTokenizer, XGLMForCausalLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq
from datasets import load_dataset


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
def load_model(base_model:str="facebook/xglm-564M"):
    tokenizer = XGLMTokenizer.from_pretrained(base_model)
    model = XGLMForCausalLM.from_pretrained(base_model,
                                           torch_dtype=torch.float16)
    return model,tokenizer
model,tokenizer = load_model()

In [6]:
new_tokens = ['<human>:', '<bot>:']

tokenizer.add_tokens(list(new_tokens))

model.resize_token_embeddings(len(tokenizer))

Embedding(256010, 1024)

In [11]:
import gdown

url = 'https://drive.google.com/uc?export=download&id=1jbbUtwgwoSQgGnXxzTh-nMReVzEU7ZTU&confirm=t&uuid=d79e2e78-51de-466f-9ceb-3944606141a2&at=AKKF8vwcgi95TGSnSQUNCKx4NTqS:1682865249145'
output = 'output.jsonl'
gdown.download(url, output, quiet=False)


Downloading...
From: https://drive.google.com/uc?export=download&id=1jbbUtwgwoSQgGnXxzTh-nMReVzEU7ZTU&confirm=t&uuid=d79e2e78-51de-466f-9ceb-3944606141a2&at=AKKF8vwcgi95TGSnSQUNCKx4NTqS:1682865249145
To: /kaggle/working/output.jsonl
100%|██████████| 167M/167M [00:01<00:00, 137MB/s] 


'output.jsonl'

In [12]:
# format data like <sep> context <human>...<bot>...
def preprocess(prompt):
    data = tokenizer(
        prompt,
        truncation=True,
        max_length=256,
        padding=False,
        return_tensors=None,
    )
    data['input_ids'].append(tokenizer.eos_token_id)
    data['attention_mask'].append(1)
    data = datacol(data)
    return data

In [13]:
from datasets import load_dataset
dataset = load_dataset('json',data_files='/kaggle/working/output.jsonl')
dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-2a44222c90c0e0ce/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['Background:', '<human>:', '<bot>:'],
        num_rows: 116288
    })
})

In [14]:
dataset = dataset['train']
dataset

Dataset({
    features: ['Background:', '<human>:', '<bot>:'],
    num_rows: 116288
})

In [15]:
from datasets import Dataset
# dataset = Dataset.from_dict(dataset['train'][:1000])
# dataset

In [16]:
def format_prompt(prompt):
    return {'prompt':f"{prompt['Background:']} {prompt['<human>:']} {prompt['<bot>:']}"}

In [17]:
dataset = dataset.map(format_prompt,remove_columns=['Background:', '<human>:', '<bot>:'])
dataset

  0%|          | 0/116288 [00:00<?, ?ex/s]

Dataset({
    features: ['prompt'],
    num_rows: 116288
})

In [18]:
# format data like <sep> context <human>...<bot>...<sep>
def preprocess(prompt):
    data = tokenizer(
        prompt['prompt'],
        truncation=True,
        max_length=256,
        padding=False,
        return_tensors=None,
    )
    data['input_ids'].append(tokenizer.eos_token_id)
    data['attention_mask'].append(1)
    data['labels'] = data['input_ids']
    return data

In [20]:
dataset = dataset.map(preprocess,remove_columns=['prompt']) 
dataset 

  0%|          | 0/116288 [00:00<?, ?ex/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 116288
})

In [23]:
dataset= dataset.filter(lambda x:x['input_ids'][0] ==2)
dataset=dataset.filter(lambda x:x['input_ids'][-1] == 2)  
dataset

  0%|          | 0/117 [00:00<?, ?ba/s]

  0%|          | 0/117 [00:00<?, ?ba/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 116288
})

In [24]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(
            tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True # ไม่รู้_
        )

In [25]:
# sample = tokenizer('hell I am your father',
#                    truncation=True,
#                    max_length=256,
#                    padding=False,
#                    return_tensors=None)
# sample['labels'] = sample['input_ids']
# sample

In [26]:
# data_collator([sample])

In [27]:
from torch.utils.data import DataLoader

batch_size = 128
mini_batch_size = 4
gradient_accumulation_steps = batch_size //mini_batch_size
print(gradient_accumulation_steps)
train_dataloader = DataLoader(
    dataset, shuffle=True, batch_size=4, collate_fn=data_collator
)
print(len(train_dataloader))
  

32


In [29]:
from transformers import AdamW,get_scheduler

optimizer = AdamW(model.parameters(), lr=3e-5)
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=100,
    num_training_steps=num_training_steps,
)
print(num_training_steps) # Learning Rate Schedules

87216


/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [30]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu") # ตรงนี้สำหรับคนใช้ GPU/CPU ในการเทรน
model.to(device)
print(device)

cuda


In [ ]:
# with torch.amp.autocast(device_type='cuda', dtype=torch.float16):
#     data = next(iter(train_dataloader))
#     data = {k:v.to(device) for k,v in data.items()}
#     print(model(**data))

In [ ]:
# train
for epoch in range(num_epochs):
    for index,data in enumerate(train_dataloader):
        data = {k:v.to(device) for k,v in data.items()}
        with torch.amp.autocast(device_type='cuda', dtype=torch.float16):
            outputs = model(**data)
            loss = outputs.loss / gradient_accumulation_steps
            loss.backward()
        
        if (index + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
        if (index+1) % (gradient_accumulation_steps**2) == 0:
            print(epoch,index+1,(index+1)%gradient_accumulation_steps,loss)
            model.save_pretrained('xglm-checkpoint') 



In [ ]:
model.save_pretrained('xglm-checkpoint') 

In [ ]:
OPTIMIZER_NAME = "optimizer.pt"
SCHEDULER_NAME = "scheduler.pt"
torch.save(optimizer.state_dict(), os.path.join("optimizer", OPTIMIZER_NAME))
torch.save(lr_scheduler.state_dict(), os.path.join("Lrscheduler", SCHEDULER_NAME))
print('save')

In [ ]:
print('end')